In [2]:
import numpy as np
import pandas as pd
import networkx as nx

from pymoo.core.problem import ElementwiseProblem
from pymoo.core.repair import Repair
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination.default import DefaultMultiObjectiveTermination
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation

In [14]:
# Filter data to limit number of nodes to 10
node_max = 15
df = pd.read_csv('bicycle_network.csv')
df = df.loc[(df.node1 < node_max) & (df.node2 < node_max)]
df['scenic_score'] = df.drop(['node1', 'node2', 'distance'], axis=1).sum(axis=1) #+ 5 * df.drop(['node1', 'node2', 'distance'], axis=1).min(axis=1)
# df['scenic_score'] = df.scenic_beauty ** 3 + df.roughness + df.safety ** 2 + df.slope * 3

g = nx.Graph()
for node1 in range(node_max):
    for node2 in range(node_max):
        if node1 == node2:
            continue
        g.add_edge(node1,
                   node2,
                   distance=df.loc[(df.node1 == node1) & (df.node2 == node2), 'distance'].values[0],
                   scenic_score=df.loc[(df.node1 == node1) & (df.node2 == node2), 'scenic_score'].values[0])




In [15]:
class BicycleRoutingProblem(ElementwiseProblem):
    def __init__(self, graph):
        super().__init__(
            n_var=node_max, n_obj=2, n_constr=0,
            xl=np.zeros(node_max), xu=np.ones(node_max) * (node_max - 1), type_var=int
        )
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        scenic_score = 0
        distance = 0
        x = np.floor(x).astype(int)
        for i in range(len(x) - 1):
            scenic_score += self.graph.get_edge_data(x[i], x[i+1])['scenic_score']
            distance += self.graph.get_edge_data(x[i], x[i+1])['distance']

        out['F'] = [distance, -scenic_score]

class FixStartPoint(Repair):
    def _do(self, problem, X, **kwargs):
        # Assuming we are always starting from node 0
        I = np.where(X == 0)[1]
        for k in range(len(X)):
            i = I[k]
            X[k] = np.concatenate([X[k, i:], X[k, :i]])
        return X


problem = BicycleRoutingProblem(g)
algorithm = NSGA2(
    pop_size=1000,
    n_offsprings=100,
    sampling=PermutationRandomSampling(),
    crossover=OrderCrossover(),
    mutation=InversionMutation(),
    eliminate_duplicates=True,
    repair=FixStartPoint()
)
termination = DefaultMultiObjectiveTermination(n_max_gen=500)

res = minimize(problem,
               algorithm,
               termination,
               save_history=True,
               verbose=True)


n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |     1000 |      7 |             - |             -
     2 |     1100 |      8 |  0.0091363011 |             f
     3 |     1200 |      9 |  0.0196132597 |         ideal
     4 |     1300 |      9 |  0.000000E+00 |             f
     5 |     1400 |      9 |  0.000000E+00 |             f
     6 |     1500 |      9 |  0.0791147291 |         ideal
     7 |     1600 |     10 |  0.0055653934 |         ideal
     8 |     1700 |     10 |  0.000000E+00 |             f
     9 |     1800 |      9 |  0.0108761924 |             f
    10 |     1900 |      9 |  0.2000000000 |         ideal
    11 |     2000 |      9 |  0.0131380518 |             f
    12 |     2100 |      9 |  0.000000E+00 |             f
    13 |     2200 |     10 |  0.0212092958 |             f
    14 |     2300 |     11 |  0.0153404586 |             f
    15 |     2400 |     11 |  0.000000E+00 |             f
    16 |     2500 |     11 |  0.000000E+00 |            

In [16]:
np.save(f'results/pareto_front_{node_max}_nodes.npy', res.F)